In [98]:
%matplotlib inline
import sys
import re
import nltk
import numpy as np
import pandas as pd
import pymorphy2
import sklearn
import itertools
from collections import Counter
from gensim.models.keyedvectors import KeyedVectors
from nltk.corpus import stopwords
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import LabelBinarizer

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input
from keras.preprocessing.text import Tokenizer
from keras import regularizers

In [ ]:
morph = pymorphy2.MorphAnalyzer()

w2v = KeyedVectors.load_word2vec_format("data/ruwikiruscorpora_0_300_20.bin", binary=True)
w2v.init_sims(replace=True)

In [4]:
data = pd.read_csv('data/250ktendersList_norm.csv', sep='\t')
data.head()

,tender_id,tender_name,lots,otrasl,podortasl,okpd2,okpd2_name,norm_name,norm_lots,norm_okpd
0,50000048,Филиал Федерального бюджетного учреждения здра...,Услуги по оценке собственности,"Бизнес, финансы, страхование, маркетинг и реклама","Бухгалтерский учет, оценка, аудит",683116120,Услуги посреднические при оценке нежилого недв...,филиал_NOUN федеральный_ADJ бюджетный_ADJ учре...,услуга_NOUN оценка_NOUN собственность_NOUN,услуга_NOUN посреднический_ADJ оценка_NOUN неж...
1,50000049,МУНИЦИПАЛЬНОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ САРГАТСКОГ...,Поставка газа,Топливо и энергетика,Газ,352000000,"Газы горючие искусственные, услуги по распреде...",муниципальный_ADJ унитарный_ADJ предприятие_NO...,поставка_NOUN газа_NOUN,газа_NOUN горючий_ADJ искусственный_ADJ услуга...
2,50000050,АКЦИОНЕРНАЯ КОМПАНИЯ 'АЛРОСА' (ПУБЛИЧНОЕ АКЦИО...,НИГП. Разработка минералого-геохимических крит...,"Строительство, недвижимость и архитектура","Проектные работы, геодезия, картография, инжен...",711239000,"Услуги в области геологических, геофизических ...",акционерный_ADJ компания_NOUN алрос_NOUN публи...,разработка_NOUN геохимический_ADJ критерий_NOU...,услуга_NOUN область_NOUN геологический_ADJ гео...
3,50000149,АКЦИОНЕРНОЕ ОБЩЕСТВО 'ТРАНСНЕФТЬ - ЦЕНТРАЛЬНАЯ...,Техническое обслуживание теплоэнергетического ...,"Офис, дом",Услуги,353012000,Услуги по снабжению паром и горячей водой по т...,акционерный_ADJ общество_NOUN транснефть_NOUN ...,технический_ADJ обслуживание_NOUN теплоэнергет...,услуга_NOUN снабжение_NOUN паром_NOUN горячий_...
4,50000153,ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ЭНЕРГЕТИКИ И ЭЛ...,Открытый запрос предложений № 476-ОЗП/16-СМП н...,"Офис, дом",Мебель,310100000,Мебель для офисов и предприятий торговли,публичный_ADJ акционерный_ADJ общество_NOUN эн...,открытый_ADJ запрос_NOUN предложение_NOUN опре...,мебель_NOUN офис_NOUN предприятие_NOUN торговл...


In [41]:
# Формируем label для отрасли.
y1_encoder = LabelBinarizer()
y1_all = y1_encoder.fit_transform(data['otrasl'])
y1_names = y1_encoder.classes_

print(y1_all[0])
for i, name in enumerate(y1_names):
    print(i, name)

[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0 IT, компьютеры, связь
1 Безопасность
2 Бизнес, финансы, страхование, маркетинг и реклама
3 Бумажное производство, тара и упаковка
4 Деревообработка, лес
5 Издательство, полиграфия
6 Легкая промышленность
7 Машиностроение
8 Медицина, фармакология
9 Металлы, металлоизделия
10 Наука, исследования, образование
11 Офис, дом
12 Перевозки, логистика, таможня
13 Продовольствие, пищевая промышленность
14 Сельское хозяйство
15 Спорт, отдых, туризм
16 Строительство, недвижимость и архитектура
17 Сырье, полуфабрикаты
18 Топливо и энергетика
19 Транспорт
20 Химия
21 Экология
22 Электротехника


In [42]:
# Формируем label для подотрасли.

data['norm_podortasl'] = data['otrasl'] + ' - ' + data['podortasl']

y2_encoder = LabelBinarizer()
y2_all = y2_encoder.fit_transform(data['norm_podortasl'])
y2_names = y2_encoder.classes_

print(y2_all[0])
for i, name in enumerate(y2_names):
    print(i, name)

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0 IT, компьютеры, связь - Web, Интернет
1 IT, компьютеры, связь - Защита информации
2 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
3 IT, компьютеры, связь - Программное обеспечение
4 IT, компьютеры, связь - Радиосвязь
5 IT, компьютеры, связь - Спутниковое телевидение
6 IT, компьютеры, связь - Телефония
7 IT, компьютеры, связь - Услуги провайдеров, хостинг, прочие услуги
8 Безопасность - Видеонаблюдение, Сигнализация, ОПС, СКД
9 Безопасность - Другое
10 Безопасность - Охрана и оборудование
11 Безопасность - Противопожарная безопасность
12 Бизнес, финансы, страхование, маркетинг и реклама - Банковское дело
13 Бизнес, финансы, страхование, маркетинг и реклама - Бухгалтерский учет, 

In [49]:
# Обучать будем подотрасль
y_all = y2_all
y_names = y2_names
y_encoder = y2_encoder

for i in range(0, 3):
    y = np.argmax(y_all[i])
    print(y, y_names[y])

13 Бизнес, финансы, страхование, маркетинг и реклама - Бухгалтерский учет, оценка, аудит
102 Топливо и энергетика - Газ
91 Строительство, недвижимость и архитектура - Проектные работы, геодезия, картография, инженерные изыскания, кадастровые работы


In [38]:
# Маски для разделения выборок
np.random.seed(1)
rand_mask = np.random.rand(len(data))
split_ratio = 0.8
mask_tr = rand_mask <= split_ratio
mask_val = rand_mask > split_ratio

y_tr = y_all[mask_tr]
y_val = y_all[mask_val]

len(y_tr), len(y_val)

(199924, 49854)

In [25]:
for i in range(0, 3):
    y = np.argmax(y_all[i])
    print(y, y_encoder.classes_[y])

13 Бизнес, финансы, страхование, маркетинг и реклама - Бухгалтерский учет, оценка, аудит
102 Топливо и энергетика - Газ
91 Строительство, недвижимость и архитектура - Проектные работы, геодезия, картография, инженерные изыскания, кадастровые работы


In [75]:
words_sets = list()
for i in range(len(data)):
    if i % 1000 == 0:
        sys.stdout.write('\rProcess: %d of %d' % (i, len(data)))
        sys.stdout.flush()
    words = set(str(data['norm_lots'][i]).split())
    words = words | set(str(data['norm_okpd'][i]).split())
    words = [w for w in words if w in w2v.vocab]
    words_sets.append(words)
print('. Complete.')

Process: 249000 of 249778. Complete.


In [78]:
for i in range(14, 16):
    y = np.argmax(y_all[i])
    print(words_sets[i])
    print(y, y_names[y])
    print()
data.loc[range(14, 16)]

49 Медицина, фармакология - Лекарственные препараты, витамины, расходные материалы
['право_NOUN', 'заключение_NOUN', 'сульфит_NOUN', 'натрий_NOUN', 'договор_NOUN', 'поставка_NOUN', 'бисульфит_NOUN']

93 Строительство, недвижимость и архитектура - Строительные материалы, оборудование и инструменты
['прочее_PRON', 'триммер_NOUN', 'назначение_NOUN', 'оборудование_NOUN', 'группировка_NOUN', 'специальный_ADJ', 'включить_VERB']



,tender_id,tender_name,lots,otrasl,podortasl,okpd2,okpd2_name,norm_name,norm_lots,norm_okpd,norm_podortasl
14,50000477,федеральное государственное унитарное предприя...,Право заключения договора на поставку бисульфи...,"Медицина, фармакология","Лекарственные препараты, витамины, расходные м...",201341120,Сульфиты,федеральный_ADJ государственный_ADJ унитарный_...,право_NOUN заключение_NOUN договор_NOUN постав...,сульфит_NOUN,"Медицина, фармакология - Лекарственные препара..."
15,50000484,Федеральное государственное унитарное предприя...,"Мотокосы, триммеры и бензорез","Строительство, недвижимость и архитектура","Строительные материалы, оборудование и инструм...",289939190,"Оборудование специального назначения прочее, н...",федеральный_ADJ государственный_ADJ унитарный_...,триммер_NOUN,оборудование_NOUN специальный_ADJ назначение_N...,"Строительство, недвижимость и архитектура - Ст..."


In [79]:
def build_words_data(mask):
    words_separated = list()
    xw = list()
    yw = list()
    print()
    for i in range(len(data)):
        if i % 1000 == 0:
            sys.stdout.write('\rProcess: %d of %d' % (i, len(data)))
            sys.stdout.flush()
        if not mask[i]:
            continue
        y_current = y_all[i]
        words = words_sets[i]
        for word in words:
            words_separated.append(word)
            xw.append(w2v.word_vec(word))
            yw.append(y_current)
    xw = np.array(xw)
    yw = np.array(yw)
    print('. Complete.')
    print(xw.shape, yw.shape)
    return words_separated, xw, yw
            
words_tr, xw_tr, yw_tr = build_words_data(mask_tr)
words_val, xw_val, yw_val = build_words_data(mask_val)


Process: 249000 of 249778. Complete.
(2551330, 300) (2551330, 131)

Process: 249000 of 249778. Complete.
(636707, 300) (636707, 131)


In [85]:
for i in range(15, 22):
    y = np.argmax(yw_tr[i])
    print(words_tr[i])
    print(y, y_names[y])
    print()

102 Топливо и энергетика - Газ
искусственный_ADJ

102 Топливо и энергетика - Газ
газа_NOUN

102 Топливо и энергетика - Газ
топливо_NOUN

102 Топливо и энергетика - Газ
трубопровод_NOUN

91 Строительство, недвижимость и архитектура - Проектные работы, геодезия, картография, инженерные изыскания, кадастровые работы
прочий_ADJ

91 Строительство, недвижимость и архитектура - Проектные работы, геодезия, картография, инженерные изыскания, кадастровые работы
разработка_NOUN

91 Строительство, недвижимость и архитектура - Проектные работы, геодезия, картография, инженерные изыскания, кадастровые работы
геохимический_ADJ



In [86]:
w_in_len = xw_tr.shape[1]
w_out_len = yw_tr.shape[1]

print(w_in_len, '->', w_out_len)

model_words_pre = Sequential()

model_words_pre.add(Dense(256, input_dim=(w_in_len)))
model_words_pre.add(Dropout(0.5))
model_words_pre.add(Activation('tanh'))

model_words_pre.add(Dense(w_out_len))
model_words_pre.add(Activation('softmax'))

model_words_pre.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

history = model_words_pre.fit(
    xw_tr, 
    yw_tr,
    batch_size=1024,
    epochs=2)

yw_pred = model_words_pre.predict(xw_val, batch_size=1024)

#model.evaluate(xw_val, yw_val, batch_size=128)

yw_true = np.argmax(yw_pred, axis=1) == np.argmax(yw_val, axis=1)
print('accuracy:', np.mean(yw_true))

300 -> 131
Epoch 1/2
2551330/2551330 [==============================] - 70s - loss: 3.2767 - acc: 0.2269    
Epoch 2/2
2551330/2551330 [==============================] - 70s - loss: 3.1651 - acc: 0.2458    
accuracy: 0.24881303331


In [87]:
for i in range(0, 1000):
    curr_val = np.argmax(yw_val[i])
    curr_pred = np.argmax(yw_pred[i])
    print(words_val[i])
    print(curr_val, y_names[curr_val])
    print(curr_pred, y_names[curr_pred])
    print()

центр_NOUN
2 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
49 Медицина, фармакология - Лекарственные препараты, витамины, расходные материалы

овд_NOUN
2 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
63 Офис, дом - Услуги

тип_NOUN
2 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
90 Строительство, недвижимость и архитектура - Полы, окна и двери

радиостанция_NOUN
2 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
2 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование

радио_NOUN
2 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
2 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование

омский_ADJ
2 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
49 Медицина

81 Спорт, отдых, туризм - Организация праздников и других мероприятий

энергетический_ADJ
94 Строительство, недвижимость и архитектура - Электроэнергия
63 Офис, дом - Услуги

электропередача_NOUN
94 Строительство, недвижимость и архитектура - Электроэнергия
94 Строительство, недвижимость и архитектура - Электроэнергия

уличный_ADJ
94 Строительство, недвижимость и архитектура - Электроэнергия
86 Строительство, недвижимость и архитектура - Дороги, мосты, тоннели

проведение_NOUN
94 Строительство, недвижимость и архитектура - Электроэнергия
56 Наука, исследования, образование - Услуги

город_NOUN
94 Строительство, недвижимость и архитектура - Электроэнергия
92 Строительство, недвижимость и архитектура - Ремонтные и строительные работы

можга_NOUN
94 Строительство, недвижимость и архитектура - Электроэнергия
49 Медицина, фармакология - Лекарственные препараты, витамины, расходные материалы

освещение_NOUN
94 Строительство, недвижимость и архитектура - Электроэнергия
94 Строительство, недви

общественный_ADJ
92 Строительство, недвижимость и архитектура - Ремонтные и строительные работы
49 Медицина, фармакология - Лекарственные препараты, витамины, расходные материалы

прочий_ADJ
92 Строительство, недвижимость и архитектура - Ремонтные и строительные работы
49 Медицина, фармакология - Лекарственные препараты, витамины, расходные материалы

совет_NOUN
92 Строительство, недвижимость и архитектура - Ремонтные и строительные работы
92 Строительство, недвижимость и архитектура - Ремонтные и строительные работы

группировка_NOUN
92 Строительство, недвижимость и архитектура - Ремонтные и строительные работы
49 Медицина, фармакология - Лекарственные препараты, витамины, расходные материалы

здание_NOUN
92 Строительство, недвижимость и архитектура - Ремонтные и строительные работы
92 Строительство, недвижимость и архитектура - Ремонтные и строительные работы

киров_NOUN
92 Строительство, недвижимость и архитектура - Ремонтные и строительные работы
92 Строительство, недвижимость и ар

In [88]:
model_words = Sequential()

model_words.add(Dense(256, input_dim=(w_in_len), 
    weights=model_words_pre.layers[0].get_weights()))
model_words.add(Activation('sigmoid'))


In [103]:
total_words_list = list(set(itertools.chain.from_iterable(words_sets)))
total_words_w2v_vectors = [w2v.word_vec(w) for w in total_words_list]
total_words_predict = model_words_pre.predict(np.array(total_words_w2v_vectors))
total_words_dict = {total_words_list[i]: total_words_predict[i] for i in range(len(total_words_list))}

In [125]:
texts_all = list()

for i in range(len(data)):
    if i % 1000 == 0:
        sys.stdout.write('\rProcess: %d of %d' % (i, len(data)))
        sys.stdout.flush()
    words_vectors = [total_words_dict[w] for w in words_sets[i]]
    if not words_vectors:
        texts_all.append(np.zeros(total_words_predict.shape[1]))
        continue
    texts_all.append(np.sum(np.array(words_vectors), axis=0))

texts_all = np.array(texts_all)

print('. Complete.')
texts_all.shape

Process: 249000 of 249778. Complete.


(249778, 131)

In [126]:
okpd_all = list()

for i in range(len(data)):
    if i % 1000 == 0:
        sys.stdout.write('\rProcess: %d of %d' % (i, len(data)))
        sys.stdout.flush()
    okpd = data['okpd2'][i]
    x = np.zeros(410)
    x[okpd//10000000] = 1;
    x[100 + okpd//100000%100] = 1;
    x[200 + okpd//1000%100] = 1;
    x[300 + okpd//10%100] = 1;
    x[400 + okpd%10] = 1;
    okpd_all.append(x)

okpd_all = np.array(okpd_all)
    
print('. Complete.')
okpd_all.shape

Process: 249000 of 249778. Complete.


(249778, 410)

In [127]:
okpd_tr = okpd_all[mask_tr]
okpd_val = okpd_all[mask_val]
print(okpd_tr.shape, okpd_val.shape)

texts_tr = texts_all[mask_tr]
texts_val = texts_all[mask_val]
print(texts_tr.shape, texts_val.shape)

print(y_tr.shape, y_val.shape)

(199924, 410) (49854, 410)
(199924, 131) (49854, 131)
(199924, 131) (49854, 131)


In [156]:
import keras.layers

okpd_input_len = okpd_tr.shape[1]
text_input_len = texts_tr.shape[1]
out_len = y_tr.shape[1]

layer_okpd_input = Input(shape=(okpd_input_len,))

layer_okpd = Dense(512, activation='relu')(layer_okpd_input)
layer_okpd = Dropout(0.5)(layer_okpd)

layer_text_input = Input(shape=(text_input_len,))

layer_text = Dense(256, activation='tanh')(layer_text_input)
layer_text = Dropout(0.5)(layer_okpd)

#layer_merged = keras.layers.Concatenate()([layer_okpd, layer_text])
#layer_merged = Dense(256, activation='relu')(layer_merged)
layer_merged = Dense(256, activation='relu')(layer_text)
layer_merged = Dropout(0.5)(layer_merged)

layer_output = Dense(out_len, activation='softmax')(layer_merged)

model = Model(inputs=[layer_okpd_input, layer_text_input], outputs=[layer_output])

model.compile(loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

history = model.fit([okpd_tr, texts_tr], y_tr,
        batch_size=128,
        epochs=3)

y_pred = model.predict([okpd_val, texts_val], batch_size=128)

model.evaluate([okpd_val, texts_val], y_val, batch_size=128)

Epoch 1/3
199924/199924 [==============================] - 23s - loss: 2.0008 - acc: 0.5346    
Epoch 2/3
199924/199924 [==============================] - 21s - loss: 1.4277 - acc: 0.6609    
Epoch 3/3
49792/49854 [============================>.] - ETA: 0s

[1.0392184186250517, 0.75592730771229388]

In [157]:
y_true = np.argmax(y_pred, axis=1) == np.argmax(y_val, axis=1)
print('accuracy:', np.mean(y_true))

accuracy: 0.755927307739


In [158]:
y_pred_all = model.predict([okpd_all, texts_all], batch_size=128)


In [159]:
y_pred_all_argmax = np.argmax(y_pred_all, axis=1)

data_out = data.copy()

data_out['old'] = data_out['norm_podortasl']
data_out['new'] = [y_names[y_pred_all_argmax[i]] for i in range(len(data_out))]

y_false = np.argmax(y_pred_all, axis=1) != np.argmax(y_all, axis=1)
data_out = data_out.loc[y_false]

dropped_columns = [
    'norm_name', 
    'norm_lots', 
    'norm_okpd', 
    'norm_podortasl',
    'otrasl',
    'podortasl',
    'okpd2']

data_out = data_out.drop(dropped_columns, axis=1)
data_out.head()

,tender_id,tender_name,lots,okpd2_name,new,old
5,50000155,Филиал 'Астраханский судоремонтный завод ' АО ...,"Доработка системы автоматического управления, ...",Услуги по ремонту прочего оборудования,"Медицина, фармакология - Медицинское оборудова...","Машиностроение - Ремонт, обслуживание, пуско-н..."
6,50000234,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ 'ОРЕН...,Поставка клапанов обратных для нужд ООО 'Оренб...,Клапаны обратные,"Машиностроение - Детали, запчасти, инструменты","Строительство, недвижимость и архитектура - Ре..."
9,50000266,АКЦИОНЕРНОЕ ОБЩЕСТВО 'ПЕРВАЯ ЭНЕРГОСЕРВИСНАЯ К...,Лот 86/1. Газоанализатор «ЭКОЛАБ»,Газоанализаторы или дымоанализаторы,"Наука, исследования, образование - Лабораторно...","Наука, исследования, образование - Услуги"
10,50000273,АКЦИОНЕРНОЕ ОБЩЕСТВО 'ДАЛЬНЕВОСТОЧНАЯ ГЕНЕРИРУ...,Установка железнодорожных весов ВТЭЦ-2,Услуги по ремонту прочего оборудования,"Медицина, фармакология - Медицинское оборудова...","Строительство, недвижимость и архитектура - Ре..."
16,50000497,ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО 'ЭЛЕКТРОПРИВОД'...,Оказание услуг по аренде испытательного оборуд...,Услуги по аренде и лизингу прочих машин и обор...,"Транспорт - Транспортные услуги, перевозки","Транспорт - Автомобили, автотранспорт, автобус..."


In [160]:
#data_out.to_csv('data/250ktendersList_false.csv', sep='\t', index=False, encoding='utf-8')

with open('data/250ktendersList_false.txt', 'wt', encoding='utf-8-sig') as f:
    for i, row in data_out.iterrows():
        f.write('id     : %08d\n' % row['tender_id'])
        f.write('tender : %s\n' % row['tender_name'])
        f.write('lots   : %s\n' % row['lots'])
        f.write('okpd2  : %s\n' % row['okpd2_name'])
        f.write('results:\n')
        f.write('  old  : %s\n' % row['old'])
        f.write('  new  : %s\n' % row['new'])
        f.write('\n')
    